In [ ]:
import numpy as np
import pandas as pd

from vpop_calibration import *

from IPython.display import display

%load_ext autoreload
%autoreload 2

In [ ]:
variable_names = ["A0", "A1"]
parameter_names = ["k_12", "k_21", "k_el", "t_clear"]


# Define the ode model
def equations(t, y, k_12, k_21, k_el, t_clear):
    ydot = [  # y[0] is A1, y[1] is A2
        k_21 * y[1] - k_12 * y[0] - (k_el * (t >= t_clear)) * y[0],
        k_12 * y[0] - k_21 * y[1],
    ]
    return ydot


def init_assignment(*params):
    return [10.0, 0.0]


pk_two_compartments_model = OdeModel(
    equations, init_assignment, variable_names, parameter_names
)

In [ ]:
nb_timesteps = 15
tmax = 24.0
time_steps = np.linspace(0.0, tmax, nb_timesteps)

log_nb_patients = 8
nb_patients = 2**log_nb_patients
print(nb_patients)
param_ranges = {
    "k_12": {"low": 0.02, "high": 0.07, "log": False},
    "k_21": {"low": 0.1, "high": 0.3, "log": False},
}

protocol_design = pd.DataFrame({"protocol_arm": ["A", "B"], "k_el": [0.1, 0.5]})
descriptors = [*param_ranges.keys(), "t_clear", "time"]
n_tasks = protocol_design.shape[0] * len(variable_names)
vpop = generate_vpop_from_ranges(log_nb_patients, param_ranges)
rng = np.random.default_rng()
vpop["t_clear"] = rng.integers(0, 24, nb_patients)
protocol_arms = ["A", "B"]
vpop = vpop.merge(
    pd.DataFrame(data=variable_names, columns=["output_name"]), how="cross"
)
vpop = vpop.merge(
    pd.DataFrame(data=protocol_arms, columns=["protocol_arm"]), how="cross"
)
display(vpop.head())

In [ ]:
dataset = pk_two_compartments_model.run_trial(vpop, protocol_design, time_steps)
dataset = dataset.rename(columns={"predicted_value": "value"})
display(dataset.head())

In [ ]:
def join_if_two(tup):
    if tup[0] == "":
        return tup[1]
    elif tup[1] == "":
        return tup[0]
    else:
        return "_".join(tup)


reshaped_df = dataset.pivot(
    index=["id"] + descriptors,
    columns=["output_name", "protocol_arm"],
    values="value",
).reset_index()
reshaped_df.columns = list(map(join_if_two, reshaped_df.columns))
display(reshaped_df.head())

In [ ]:
# initiate our GP class
myGP = GP(
    dataset,
    descriptors,
    var_strat="IMV",  # either IMV (Independent Multitask Variational) or LMCV (Linear Model of Coregionalization Variational)
    kernel="RBF",  # Either RBF or SMK
    deep_kernel=False,
    data_already_normalized=False,  # default
    nb_inducing_points=100,
    mll="ELBO",  # default, otherwise PLL
    nb_training_iter=200,
    training_proportion=0.7,
    learning_rate=0.1,
    lr_decay=0.99,
    jitter=1e-6,
    log_inputs=[],
    log_outputs=["A0", "A1"],
)

In [ ]:
myGP.train()

In [ ]:
myGP.plot_obs_vs_predicted("training")

In [ ]:
myGP.plot_all_solutions("training")